# Feux de forêt

Projet Python de 2A à l'ENSAE portant sur l'étude des feux de forêt en France.

In [62]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import urllib
import os
from datetime import datetime

# I. Récupération et traitement des données

On récupère les données sur le site de Météo-France. Les données disponibles complètes sur l'année sont de 1997 à 2019.
https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=90&id_rubrique=32

Voici notre fonction pour télécharger les données :

In [24]:
def download_data_meteo():
    
    for k in range(1997,2020):
        for i in range(1,13):

            if i <= 9:
                month = '0' + str(i)
            else:
                month = str(i)

            file_date = str(k) + month
            file_url = 'https://donneespubliques.meteofrance.fr/donnees_libres/Txt/Synop/Archive/synop.' + file_date + '.csv.gz'
            output_file_name = file_date + '.csv.gz'

            if os.path.exists('bdd_meteo') == 0:
                os.mkdir('bdd_meteo') 

            urllib.request.urlretrieve (file_url, 'bdd_meteo/' + output_file_name)
    
    return 'Téléchargement terminé !'    

Test d'un fichier en ouverture :

In [29]:
pd.read_csv('bdd_meteo/200202.csv.gz', sep = ';')

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4,Unnamed: 59
0,7005,20020201000000,101540,20,3,210,7.200000,281.850000,280.650000,92,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
1,7015,20020201000000,101520,-20,5,200,8.700000,281.450000,279.450000,87,...,7,3,3000,mq,mq,mq,mq,mq,mq,NaN
2,7020,20020201000000,101500,180,2,220,9.800000,283.650000,283.050000,96,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
3,7027,20020201000000,101770,380,1,240,5.100000,282.450000,280.550000,88,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
4,7037,20020201000000,101690,10,3,250,8.200000,282.450000,281.850000,96,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12891,78922,20020228210000,101350,-70,5,80,4.100000,298.750000,290.650000,61,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
12892,81401,20020228210000,100980,mq,mq,50,5.100000,300.750000,296.350000,77,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
12893,81405,20020228210000,100970,mq,mq,70,4.100000,299.750000,297.050000,85,...,4,8,660,5,3,3000,mq,mq,mq,NaN
12894,81415,20020228210000,100960,mq,mq,mq,mq,298.150000,296.950000,93,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN


Nous voyons que les fichiers bruts ont énormément de colonnes avec des noms peu clairs. Il y a également beaucoup de NaN (ici 'mq') que nous devrons traiter.

Voici la fonction pour traiter un fichier :

In [177]:
def cleaned_csv(year, month):
    
    if month <= 9:
        file = 'bdd_meteo/' + str(year) + '0' + str(month) + '.csv.gz'
    
    else:
        file = 'bdd_meteo/' + str(year) + str(month) + '.csv.gz'
        
    #ouvrir le fichier   
    df_file = pd.read_csv(file, sep=';')
    
    #sélectionner et renommer les colonnes
    df_file = df_file.rename(columns={'numer_sta':'Station', 'date':'Date', 't':'Température (°C)', 'rr3':'Précipitations (3 heures)', 'u':'Humidité (%)', 'ff':'Vitesse du vent (m/s)'})
    df_file = df_file[['Station', 'Date', 'Température (°C)', 'Précipitations (3 heures)', 'Humidité (%)', 'Vitesse du vent (m/s)']]
    
    #modifier le format de la date et l'heure
    df_file['Date'] = df_file['Date'].apply(lambda x: datetime(year = int(str(x)[0:4]), month = int(str(x)[4:6]), day = int(str(x)[6:8]), hour = int(str(x)[8:10])))
    
    #convertir les Kelvin en degrés Celsius
    l = list(df_file['Température (°C)'])
    for i in range(len(l)):
        if l[i] != 'mq':
            df_file.loc[i, 'Température (°C)'] = float(l[i]) - 273.15
    
    #trier par station et date ainsi que reset l'index
    df_file = df_file.sort_values(['Station', 'Date']).reset_index(drop = True)
    
    return df_file

Voici un exemple du résultat :

In [178]:
cleaned_csv(2010,11)

,Station,Date,Température (°C),Précipitations (3 heures),Humidité (%),Vitesse du vent (m/s)
0,7005,2010-11-01 00:00:00,6.7,0.000000,94,2.600000
1,7005,2010-11-01 03:00:00,6.2,0.200000,95,2.600000
2,7005,2010-11-01 06:00:00,8.7,0.000000,95,1.500000
3,7005,2010-11-01 09:00:00,10.5,0.000000,95,3.100000
4,7005,2010-11-01 12:00:00,11.5,0.000000,92,5.100000
...,...,...,...,...,...,...
13938,89642,2010-11-30 09:00:00,-2,mq,33,1.500000
13939,89642,2010-11-30 12:00:00,-5.6,mq,29,3.600000
13940,89642,2010-11-30 15:00:00,-8.3,mq,32,1.000000
13941,89642,2010-11-30 18:00:00,-10.1,mq,42,2.600000


Il ne reste plus qu'à regrouper tous les fichiers mensuels en un seul dataframe :

In [185]:
df_meteo = pd.DataFrame()

for k in range(2019,2020):
    for i in range(1,13):       
        df_meteo = pd.concat([df_meteo, cleaned_csv(k,i)], ignore_index = True)

df_meteo

,Station,Date,Température (°C),Précipitations (3 heures),Humidité (%),Vitesse du vent (m/s)
0,7005,2019-01-01 00:00:00,8.1,0.000000,87,2.200000
1,7005,2019-01-01 03:00:00,8.2,0.000000,84,3.400000
2,7005,2019-01-01 06:00:00,7.7,0.000000,82,3.300000
3,7005,2019-01-01 09:00:00,7.4,0.000000,83,2.600000
4,7005,2019-01-01 12:00:00,8.8,0.000000,81,4.200000
...,...,...,...,...,...,...
173151,89642,2019-12-31 00:00:00,0.7,mq,62,28.300000
173152,89642,2019-12-31 12:00:00,1.7,mq,74,23.100000
173153,89642,2019-12-31 15:00:00,1,mq,78,18.000000
173154,89642,2019-12-31 18:00:00,1,mq,72,20.600000


In [186]:
df_meteo.head(50)

,Station,Date,Température (°C),Précipitations (3 heures),Humidité (%),Vitesse du vent (m/s)
0,7005,2019-01-01 00:00:00,8.1,0.000000,87,2.200000
1,7005,2019-01-01 03:00:00,8.2,0.000000,84,3.400000
2,7005,2019-01-01 06:00:00,7.7,0.000000,82,3.300000
3,7005,2019-01-01 09:00:00,7.4,0.000000,83,2.600000
4,7005,2019-01-01 12:00:00,8.8,0.000000,81,4.200000
5,7005,2019-01-01 15:00:00,9,0.000000,88,4.200000
6,7005,2019-01-01 18:00:00,8.3,0.000000,92,2.200000
7,7005,2019-01-01 21:00:00,7.3,0.000000,82,3.200000
8,7005,2019-01-02 00:00:00,5.2,-0.100000,86,2.100000
9,7005,2019-01-02 03:00:00,4.2,-0.100000,85,3.100000


A faire : importer les bdd feux et carte de France en s'inspirant du code ci-dessus